In [44]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime

In [45]:
day = datetime.today()

In [46]:
day

datetime.datetime(2020, 6, 29, 3, 47, 31, 589750)

# Функции 

#### Head Hunter

In [47]:
def parse_head_hunter(page, position, vacancy_hh = []):
    """
    В этой функии мы парсим страницы сайта hh
    
    page: передаем список с номерами страниц и после проверки будем его переводить во множество, 
          чтобы исключить повторения
          
          
    position: Передаем должность, по которой будем собирать данные  
    
    """
    
    
    main_link = 'https://hh.ru/'


    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                            AppleWebKit/537.36 (KHTML, like Gecko)\
                            Chrome/83.0.4103.116 Safari/537.36'}
    
    
    
    # Список для хранения данных по должности 
    
    
    
    # Перебираем все страницы 

        
    params = {'text':position,
                 'area':1,
                  'L_save_area':True,
                 'clusters':True,
                 'enable_snippets':True,
                  'st':'searchVacancy',
                 'fromSearch':True,
                 'from':'suggest_post',
                  'customDomain':1,
                  'page':page-1
                 }

    html = requests.get(main_link + 'search/vacancy/' , headers = headers, params = params)

    soup = bs(html.text,'html.parser')

    vacancy_block = soup.find('div',{'class':'vacancy-serp'})

    vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})


    for i in vacancy_list:
        vacancy_dict = {}
        vacancy_name = i.find('a',{'class':'bloko-link'}).getText()
        vacancy_link = i.find('a',{'class':'bloko-link'})['href']
        vacancy_salary = i.find('div',{'class':'vacancy-serp-item__sidebar'}).getText()
        vacancy_employer = i.find('div',{'class':'vacancy-serp-item__meta-info'}).getText()
        vacancy_city = i.find('span',{'class':'vacancy-serp-item__meta-info'}).getText().split(', ')
        vacancy_date = i.find('span',{'class':'vacancy-serp-item__publication-date'}).getText()



        currency = vacancy_salary[-4:] # выводим валюту
        if not currency:
            currency = np.nan
        else:
            vacancy_salary = vacancy_salary.replace(currency,'') # удаляем валюту


        #Проверка метро, если указано метро. то мы его найдем при попощи проверки
        # если его нет , то ставим NaN
        if len(vacancy_city)>1:
            vacancy_metro = vacancy_city[1]
        else:
            vacancy_metro  = np.nan


        # Делаем разделение на минимальную и максимальную ЗП. 
        # Если где-то не указана сумма ЗП за то мы заполняем значением NaN
        if '-' in vacancy_salary:

            salaty_split = vacancy_salary.split('-')
            salary_min = salaty_split[0].replace(' ','')
            salary_max = salaty_split[1].replace(' ','')

        elif 'от' in vacancy_salary:

            salaty_rep = vacancy_salary.replace('от ','')
            salary_min = salaty_rep
            salary_max = np.nan

        elif 'до' in vacancy_salary:

            salaty_rep = vacancy_salary.replace('до ','')
            salary_min = np.nan
            salary_max = salaty_rep

        else:
            salary_min = np.nan
            salary_max = np.nan




        vacancy_dict['vacancy_employer'] = vacancy_employer
        vacancy_dict['vacancy_name'] = vacancy_name
        vacancy_dict['salary_min'] = salary_min
        vacancy_dict['salary_max'] = salary_max
        vacancy_dict['currency'] = currency
        vacancy_dict['vacancy_city'] = vacancy_city[0]
        vacancy_dict['vacancy_date'] = vacancy_date
        vacancy_dict['vacancy_link'] = vacancy_link[:vacancy_link.index('?')]


        vacancy_hh.append(vacancy_dict)
        
        
    return vacancy_hh

#### SuperJob

In [48]:
def parse_super_job(page, position,vacancy_sj = []): 
    
    """
    В этой функии мы парсим страницы сайта SJ
    
    page: передаем множесво с номерами страниц, 
          чтобы не повторялимь значения и мы не собирали одинаковые данные
          
    position: Передаем должность, по которой будем собирать данные  
    
    """
    

    
    main_link_sj = 'https://superjob.ru'

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                            AppleWebKit/537.36 (KHTML, like Gecko)\
                            Chrome/83.0.4103.116 Safari/537.36'}
    
    
        
    params = {'keywords':position,
                 'page': page}

    html_sj = requests.get(main_link_sj+'/vacancy/search/', headers = headers, params = params)

    soup = bs(html_sj.text,'html.parser')

    vacancy_block = soup.find('div',{'class':'_1ID8B'})

    vacancy_list = vacancy_block.find_all('div',{'class':'Fo44F'})
        
        


        

    for i in vacancy_list:
        vacancy_dict = {}

        vacancy_name = i.find('div',{'class':'_3mfro'}).getText()
        vacancy_link = i.find('a',{'class':'icMQ_'})['href']
        vacancy_employer = i.find_all('a',{'class':'icMQ_'})[1].getText()
        vacancy_salary = i.find_all('span',{'class':'_3mfro'})[0].getText()
        vacancy_city = i.find_all('span',{'class':'_9fXTd'})[1].getText()




        if 'По дого' in vacancy_salary:
            currency = np.nan
        else:
            currency = vacancy_salary[-4:] # выводим валюту
            vacancy_salary = vacancy_salary.replace(currency,'') # удаляем валюту   




        if '—' in vacancy_salary:
            salaty_split = vacancy_salary.split('—')
            salary_min = salaty_split[0]
            salary_max = salaty_split[1]
        elif 'от\xa0' in vacancy_salary:
            salaty_rep = vacancy_salary.replace('от\xa0','')
            salary_min = salaty_rep
            salary_max = np.nan
        elif 'до\xa0' in vacancy_salary:
            salaty_rep = vacancy_salary.replace('до\xa0','')
            salary_min = np.nan
            salary_max = salaty_rep
        else:
            salary_min = np.nan
            salary_max = np.nan





        vacancy_date = vacancy_city.split(' • ')[0]
        vacancy_metro = np.nan

        if len(vacancy_city.split(' • ')) == 1:
            vacancy_city = [np.nan]

        elif len(vacancy_city.split(' • ')) == 2:
            vacancy_city = vacancy_city.split(' • ')[1].split(', ')
                

        vacancy_dict['vacancy_employer'] = vacancy_employer
        vacancy_dict['vacancy_name'] = vacancy_name
        vacancy_dict['salary_min'] = salary_min
        vacancy_dict['salary_max'] = salary_max
        vacancy_dict['currency'] = currency
        vacancy_dict['vacancy_city'] = vacancy_city[0]
        vacancy_dict['vacancy_date'] = vacancy_date
        vacancy_dict['vacancy_link'] = main_link_sj+vacancy_link



        vacancy_sj.append(vacancy_dict)
    
    return vacancy_sj

#### Возможные страницы для поиска

In [49]:
def max_number_page_SJ(number, position):
    
    main_link_sj = 'https://superjob.ru'

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                            AppleWebKit/537.36 (KHTML, like Gecko)\
                            Chrome/83.0.4103.116 Safari/537.36'}
    vacancy_sj = []

        
    params = {'keywords':position}

    html_sj = requests.get(main_link_sj+'/vacancy/search/', headers = headers, params = params)

    soup = bs(html_sj.text,'html.parser')
    
    count_block = soup.find('div',{'class':'L1p51'})
    
    if count_block:

        max_count_list = count_block.find_all('span',{'class':'_3IDf-'})[-2].getText()
    else:
        return 1

    return np.arange(1,int(max_count_list)+1)

In [50]:
def max_number_page_HH(number, position):
    
    main_link_hh = 'https://hh.ru/'

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                            AppleWebKit/537.36 (KHTML, like Gecko)\
                            Chrome/83.0.4103.116 Safari/537.36'}



    params = {'text':position,
             'area':1,
              'L_save_area':True,
             'clusters':True,
             'enable_snippets':True,
              'st':'searchVacancy',
             'fromSearch':True,
             'from':'suggest_post',
              'customDomain':1,
              'page':number-1
             }
    
    html = requests.get(main_link_hh + 'search/vacancy/' , headers = headers, params = params)

    soup = bs(html.text,'html.parser')

    count_block = soup.find('div',{'data-qa':'pager-block'})
    
    if count_block:
        
        max_count_list = count_block.find_all('a',{'class':'bloko-button'})[-2].getText()
        
    else:
        
        return 1
    

    return np.arange(1,int(max_count_list)+1)

#### DataSet

In [51]:
def df_set_all(value_1, value_2):
    
    """
        Функция сделана для преобразование и объединения DataSet 
        настройка индексов и преобразование зарплаты в тип int
        Делаем переименование столбцов и добавляем сайт , откуда был собран материал 
    
    """
    
    df_1 = pd.DataFrame(value_1)
    
    df_2 = pd.DataFrame(value_2)
    
    df = pd.concat((df_1, df_2))
    
    df['website'] = 'www.'+df['vacancy_link'].str.extract("(\w+)\.")+'.ru'
    
    df = df.rename(columns = {'vacancy_employer':'Работодатель',
         'vacancy_name':'Вакансия',
         'salary_min':'Зарплата_от',
         'salary_max':'Зарплата_до',
         'vacancy_city':'Город',
         'vacancy_date':'Дата_публикации',
         'vacancy_link':'Ссылка_на_вакансию',
          'website':'Сайт',
            'currency':'Валюта'})
    
    df = df.reset_index().drop('index',axis=1)
    
    
    
    df.loc[df['Зарплата_от'].isnull(), 'Зарплата_от'] = '0'
    df.loc[df['Зарплата_до'].isnull(), 'Зарплата_до'] = '0'
    
    df['Зарплата_от'] = df['Зарплата_от'].str.replace('\xa0','').astype(int)
    df['Зарплата_до'] = df['Зарплата_до'].str.replace('\xa0','').astype(int)
    
    df.style.highlight_null(null_color='red')
    
    
    return df

### Должность 

#### Введите должность, напрмиер Аналитик

In [52]:
position = input('Введите должность: ')

Введите должность: Аналитик


### Сбор данных c HeadHunter

In [53]:
my_pages_hh = max_number_page_HH(1,position)

In [54]:
print(f'Страницы для поиска на hh.ru: {my_pages_hh}\n')
print()

while True:
    
    pages_hh = np.array(input('(Введите страницы через пробел или одну страницу)\nС каких страниц собираем данные? hh.ru: ').split()).astype(int)
    
    for i in range(len(pages_hh)):
        if pages_hh[i] == 0:
            pages_hh[i] = 1
    
    
    if pages_hh.size == 0:
        continue
    
    count_false = 0

    for number_page_hh in pages_hh:

        if np.in1d(number_page_hh, my_pages_hh):

            my_data_hh = parse_head_hunter(number_page_hh,position)

            my_pages_hh = np.setdiff1d([my_pages_hh],number_page_hh)
        else:
            count_false+=1
            
    if count_false == pages_hh.size:
        
        print('Указаны страницы, которых нет или уже проверена')
        print()
        
        no_or_yes = input('Продолжить парсинг? y/n: ')

        while (no_or_yes != 'y') and (no_or_yes !='n'):
            
            print('Не корректный ввод')
            print()
            
            no_or_yes = input('Продолжить парсинг? y/n: ')

        if no_or_yes == 'y':
            
            continue
        else:
            
            break
            
    print(f'Оставшиеся страницы для парсинга: {my_pages_hh}')
    
    if my_pages_hh.size != 0:
        
        no_or_yes = input('Продолжить парсинг? y/n: ')

        while (no_or_yes != 'y') and (no_or_yes !='n'):
            
            print('Не корректный ввод')
            print()
            
            no_or_yes = input('Продолжить парсинг? y/n: ')

        if no_or_yes == 'y':
            
            continue
        else:
            
            break
    else:
        
        print('Страниц для парсинга не осталось')
        
        break

Страницы для поиска на hh.ru: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]


(Введите страницы через пробел или одну страницу)
С каких страниц собираем данные? hh.ru: 1 2 3
Оставшиеся страницы для парсинга: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40]
Продолжить парсинг? y/n: n


### Сбор данных c SuperJob

In [55]:
my_pages_sj = max_number_page_SJ(1,position)

In [56]:
print(f'Страницы для парсинга на superjob.ru: {my_pages_sj}')
print()

while True:
    
    pages_sj = np.array(input('(Введите страницы через пробел или одну страницу)\nС каких страниц собираем данные? superjob.ru: ').split()).astype(int)
    
    for i in range(len(pages_sj)):
        if pages_sj[i] == 0:
            pages_sj[i] = 1

    
    if pages_sj.size == 0:
        continue
    
    count_false = 0

    for number_page_sj in pages_sj:

        if np.in1d(number_page_sj, my_pages_sj):

            my_data_sj = parse_super_job(number_page_sj,position)

            my_pages_sj = np.setdiff1d([my_pages_sj],number_page_sj)
        else:
            count_false+=1
            
    if count_false == pages_sj.size:
        
        print('Указаны страницы, которых нет или уже проверена Вот тутн')
        print()
        
        no_or_yes = input('Продолжить парсинг? y/n: ')

        while (no_or_yes != 'y') and (no_or_yes !='n'):
            
            print('Не корректный ввод')
            print()
            
            no_or_yes = input('Продолжить парсинг? y/n: ')

        if no_or_yes == 'y':
            
            continue
        else:
            
            break
            
    print(f'Оставшиеся страницы для парсинга: {my_pages_sj}')
    print()
    
    if my_pages_sj.size != 0:
        
        no_or_yes = input('Продолжить парсинг? y/n: ')

        while (no_or_yes != 'y') and (no_or_yes !='n'):
            
            print('Не корректный ввод')
            print()
            
            no_or_yes = input('Продолжить парсинг? y/n: ')

        if no_or_yes == 'y':
            
            continue
        else:
            
            break
    else:
        
        print('Страниц для парсинга не осталось, данные собраны')
        print()
        
        break

Страницы для парсинга на superjob.ru: [1 2 3 4]

(Введите страницы через пробел или одну страницу)
С каких страниц собираем данные? superjob.ru: 1
Оставшиеся страницы для парсинга: [2 3 4]

Продолжить парсинг? y/n: n


# Dataset 

In [57]:
my_df = df_set_all(my_data_hh, my_data_sj)

In [58]:
my_df.groupby('Сайт')['Ссылка_на_вакансию'].count()

Сайт
www.hh.ru          150
www.superjob.ru     20
Name: Ссылка_на_вакансию, dtype: int64

In [59]:
my_df

,Работодатель,Вакансия,Зарплата_от,Зарплата_до,Валюта,Город,Дата_публикации,Ссылка_на_вакансию,Сайт
0,ООО Your Suit,Руководитель отдела продаж / Директор по продажам,130000,250000,руб.,Москва,28 июня,https://hh.ru/vacancy/37713537,www.hh.ru
1,Alphaopen,Бизнес-аналитик программного решения,160000,220000,руб.,Москва,26 июня,https://hh.ru/vacancy/37694581,www.hh.ru
2,NodaSoft,MSK.IT.ANALIST,120000,0,руб.,Москва,28 июня,https://hh.ru/vacancy/37715080,www.hh.ru
3,ООО Банкирро,Ведущий менеджер по продажам,170000,210000,руб.,Москва,26 июня,https://hh.ru/vacancy/37229320,www.hh.ru
4,Банк ВТБ (ПАО),"Стажер ВТБ IT-Юниор (Аналитика, разработка, De...",0,0,NaN,Москва,28 июня,https://hh.ru/vacancy/37595281,www.hh.ru
...,...,...,...,...,...,...,...,...,...
165,НИИОЗММ ДЗМ,Аналитик (визионер),0,0,NaN,Москва,Вчера,https://superjob.ru/vakansii/analitik-32380682...,www.superjob.ru
166,Эвотор,Аналитик SQL (DMP),0,0,NaN,Москва,Вчера,https://superjob.ru/vakansii/analitik-sql-3402...,www.superjob.ru
167,"АНО Радиоканал ""Вера, надежда, любовь""",CRM-менеджер (Аналитик),60000,0,руб.,Москва,27 июня,https://superjob.ru/vakansii/crm-menedzher-339...,www.superjob.ru
168,Sandwichbar,Аналитик продаж,65000,80000,руб.,Москва,27 июня,https://superjob.ru/vakansii/analitik-prodazh-...,www.superjob.ru
